## Pedestrian detection


## Dataset contains images of pedestrians and images with no pedestrians

## Importing libraries

In [20]:
import os
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

%matplotlib inline

## Loading the dataset

### Resizing the images

In [6]:
import cv2
import os

datapath = 'C:\\Users\\Downloads\\Padestrian\\data\\train\\'
files_path = os.listdir(datapath)

for i in files_path:
    path_final = 'C:\\Users\\Downloads\\Padestrian\\data\\train\\' + i + '\\'
    final = os.listdir(path_final)
    for j in final:
        data = path_final + j
        img = cv2.imread(data)
        image = cv2.resize(img, (128,128))
        cv2.imwrite(data,image)



In [7]:
import cv2
import os

datapath = 'C:\\Users\\Downloads\\Padestrian\\data\\validation\\'
files_path = os.listdir(datapath)

for i in files_path:
    path_final = 'C:\\Users\\Downloads\\Padestrian\\data\\validation\\' + i + '\\'
    final = os.listdir(path_final)
    for j in final:
        data = path_final + j
        img = cv2.imread(data)
        image = cv2.resize(img, (128,128))
        cv2.imwrite(data,image)


In [2]:
data = 'C:\\Users\\Downloads\\Padestrian\\data'
test = data+'\\validation\\'
train = data+'\\train\\'


## Now, we are using the image data generator function

In [3]:
image_shape = (128,128,3)
image_gen = ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.05,
                               height_shift_range=0.05,
                               rescale=1/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest',
                              vertical_flip = True)

## Building a convolutional neural network model 1

In [4]:
model = Sequential()
model.add(Conv2D(filters=256, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 256)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 128)       295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 64)        73792     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 64)       0

### Loading images from memory for training the model and validating the model

In [11]:

batch_size=64
train_image_gen = image_gen.flow_from_directory(train,
                                               target_size= (128,128),
                                                
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 1262 images belonging to 2 classes.


In [12]:
val_image_gen = image_gen.flow_from_directory(test,
                                               target_size= (128,128),
                                              
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 354 images belonging to 2 classes.


In [13]:
print(train_image_gen.class_indices)
Labels=train_image_gen.class_indices
print(val_image_gen.class_indices)

{'no pedestrian': 0, 'pedestrian': 1}
{'no pedestrian': 0, 'pedestrian': 1}


### Training the model 1

In [14]:

results = model.fit(train_image_gen,epochs=20,
                              validation_data=val_image_gen)

Epoch 1/20
20/20 [==============================] - 133s 7s/step - loss: 0.6878 - accuracy: 0.5269 - val_loss: 0.6872 - val_accuracy: 0.6808
Epoch 2/20
20/20 [==============================] - 128s 6s/step - loss: 0.6830 - accuracy: 0.5729 - val_loss: 0.6634 - val_accuracy: 0.5339
Epoch 3/20
20/20 [==============================] - 130s 7s/step - loss: 0.6014 - accuracy: 0.6949 - val_loss: 0.6049 - val_accuracy: 0.6723
Epoch 4/20
20/20 [==============================] - 128s 6s/step - loss: 0.5171 - accuracy: 0.7409 - val_loss: 0.5357 - val_accuracy: 0.7203
Epoch 5/20
20/20 [==============================] - 124s 6s/step - loss: 0.5802 - accuracy: 0.7060 - val_loss: 0.5671 - val_accuracy: 0.6808
Epoch 6/20
20/20 [==============================] - 127s 6s/step - loss: 0.4965 - accuracy: 0.7615 - val_loss: 0.5028 - val_accuracy: 0.7373
Epoch 7/20
20/20 [==============================] - 132s 7s/step - loss: 0.4659 - accuracy: 0.7750 - val_loss: 0.4842 - val_accuracy: 0.7655
Epoch 8/20
20

### Predicting the validation dataset

In [17]:
predictions = np.argmax(model.predict(val_image_gen), axis = -1)
predictions

6/6 [==============================] - 9s 1s/step


array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

### Evaluating the model 1

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(val_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.73      0.86      0.79       177
           1       0.83      0.68      0.75       177

    accuracy                           0.77       354
   macro avg       0.78      0.77      0.77       354
weighted avg       0.78      0.77      0.77       354



### Building the Convolutional neural network model 2

In [25]:
import tensorflow as tf
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation = 'sigmoid'))
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 126, 126, 128)     3584      
                                                                 
 conv2d_8 (Conv2D)           (None, 124, 124, 64)      73792     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 62, 62, 64)        0         
                                                                 
 flatten_3 (Flatten)         (None, 246016)            0         
                                                                 
 dense_6 (Dense)             (None, 64)                15745088  
                                                      

### Training the model 2

In [29]:
results = model.fit(train_image_gen,epochs=12,
                              validation_data=val_image_gen)

Epoch 1/12
20/20 [==============================] - 136s 7s/step - loss: 0.5477 - accuracy: 0.7361 - val_loss: 0.5831 - val_accuracy: 0.6723
Epoch 2/12
20/20 [==============================] - 142s 7s/step - loss: 0.5346 - accuracy: 0.7504 - val_loss: 0.5473 - val_accuracy: 0.7345
Epoch 3/12
20/20 [==============================] - 135s 7s/step - loss: 0.5116 - accuracy: 0.7662 - val_loss: 0.5356 - val_accuracy: 0.7373
Epoch 4/12
20/20 [==============================] - 135s 7s/step - loss: 0.5064 - accuracy: 0.7662 - val_loss: 0.5280 - val_accuracy: 0.7514
Epoch 5/12
20/20 [==============================] - 138s 7s/step - loss: 0.4941 - accuracy: 0.7734 - val_loss: 0.5163 - val_accuracy: 0.7571
Epoch 6/12
20/20 [==============================] - 137s 7s/step - loss: 0.4959 - accuracy: 0.7789 - val_loss: 0.5897 - val_accuracy: 0.7119
Epoch 7/12
20/20 [==============================] - 136s 7s/step - loss: 0.4965 - accuracy: 0.7789 - val_loss: 0.5070 - val_accuracy: 0.7599
Epoch 8/12
20

### Doing the prediction over validation dataset

In [30]:
predictions = np.argmax(model.predict(val_image_gen), axis = -1)
predictions

6/6 [==============================] - 9s 1s/step


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

### Evaluating performance of model 2

In [31]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(val_image_gen.classes,predictions))

              precision    recall  f1-score   support

           0       0.76      0.75      0.76       177
           1       0.75      0.76      0.76       177

    accuracy                           0.76       354
   macro avg       0.76      0.76      0.76       354
weighted avg       0.76      0.76      0.76       354

